# Парсинг

In [22]:
# !pip install requests

Для проекта мы хотим собрать данные о винах из каталога интернет-магазина SimpleWine.

In [23]:
import requests

# устанавливаем соединение с сайтом
url = 'https://simplewine.ru/catalog/vino/page1/.html'
response = requests.get(url)
response

<Response [200]>

Для парсинга используем библиотеку BeautifulSoup.

In [24]:
from bs4 import BeautifulSoup

# дерево исследуемой веб-страницы
tree = BeautifulSoup(response.content, 'html.parser')

Изучив код страницы, обнаруживаем, что информация об отдельном вине в каталоге лежит внутри тега `article`, для которого прописан класс `snippet swiper-slide`.



In [25]:
# собираем информацию о всех винах с исследуемой страницы
wines = tree.find_all('article', {'class': 'snippet swiper-slide'})

In [26]:
len(wines)

32

На одной странице каталога представлено 32 вида вина.

Большую часть инстересующих нас характиристик (страна производства, цвет, сладость, объём, виды винограда, производитель, регион производства) можно достать из тега `div`, для которого прописан класс `snippet-description--big`.

In [27]:
data = wines[0].find('div', {'class': 'snippet-description--big'}).find_all('a')
data

[<a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-1" href="/catalog/vino/filter/country-yuzhnaya_afrika/">Южная Африка</a>,
 <a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-2" href="/catalog/vino/filter/color-krasnoe/">красное</a>,
 <a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-3" href="/catalog/vino/filter/sugar_type-sukhoe/">сухое</a>,
 <a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-4" href="/catalog/vino/filter/volume-0_75/">0.75 л.</a>,
 <a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-5" href="/catalog/vino/filter/grape-shiraz/">шираз 100%</a>,
 <a data-autotest-target="product-item-big-snippet-1" data-autotest-target-id="product-item-snippet-1-filter-6" href="/catalog/vino/filter/manu

Достанем все эти характеристики и запишем их в соответствующие переменные.

In [28]:
country, color, sugar, volume, grape, manufacturer, region = [el.text.strip() for el in data]
country, color, sugar, volume, grape, manufacturer, region

('Южная Африка',
 'красное',
 'сухое',
 '0.75\xa0л.',
 'шираз 100%',
 'Simonsig',
 'Стелленбош')

Информацию о цене можно найти внутри тега `div`, для которого прописан класс `snippet-price`. В классе `snippet-price` есть ещё один тег `div`, в котором уже непосредственно лежит информация о цене вина.

In [29]:
price = wines[0].find('div', {'class': 'snippet-price'}).div.text.split('₽')[0].strip()
price

'2 990'

Аналогично находим информацию о годе производства и названии вина на странице.

In [30]:
year = wines[0].find('div', {'id': 'snippet-buy-block'}).get('data-product-year')
year

'2020\xa0г.'

In [31]:
name = wines[0].find('div',  {'id': 'snippet-buy-block'}).get('data-product-name')
name

"Shiraz Mr Borio's"

Напишем функцию, собирающую информацию о винах с одной страницы каталога.

In [32]:
def get_page(p, output):
  # ссылка на страницу каталога
  url = f'https://simplewine.ru/catalog/vino/page{p}/.html'
  response = requests.get(url)
  # дерево с информацией о товарах
  tree = BeautifulSoup(response.content, 'html.parser')
  wines = tree.find_all('article', {'class': 'snippet swiper-slide'})

  # сбор информации о товарах
  for wine in wines:
    name = wine.find('div',  {'id': 'snippet-buy-block'}).get('data-product-name')

    price_info = wine.find('div', {'class': 'snippet-price'})
    if price_info is not None:  # проверяем наличие целевой переменной
      price = price_info.div.text.split('₽')[0].strip()

      features = wine.find('div', {'class': 'snippet-description--big'}).find_all('a')
      year_info = wine.find('div', {'id': 'snippet-buy-block'})
      # проверяем наличие всех основных характеристик
      if len(features) >= 7 and year_info is not None:
        country, color, sugar, volume = [el.text.strip() for el in features[:4]]
        manuf, region = [el.text.strip() for el in features[-2:]]
        grape = '&'.join([el.text.strip() for el in features[4:-2]])
        year = year_info.get('data-product-year')
        # записываем в файл информацию о товаре
        print(f'{name},{country},{color},{sugar},{volume},{manuf},{region},{grape},{year},{price}\n',
              file=output)

Всего в каталоге 177 страниц. Пройдемся в цикле по всем страницам каталога, записывая в файл 'data.csv' информацию о винах. Для сбора информации используем ранее написанную функцию.

In [35]:
with open('data.csv', 'w') as output:
  print('name,country,color,sugar,volume,manufacturer,region,grape,year,price\n',
              file=output) # названия колонок
  for p in range(1, 178):
    get_page(p, output)

Выгрузим из полученного файла данные в таблицу.

In [38]:
import pandas as pd

df = pd.read_csv('data.csv', on_bad_lines='skip')
print(df.shape)
df.head()

(5571, 10)


,name,country,color,sugar,volume,manufacturer,region,grape,year,price
0,Shiraz Mr Borio's,Южная Африка,красное,сухое,0.75 л.,Simonsig,Стелленбош,шираз 100%,2020 г.,2 990
1,Gavi dei Gavi (Etichetta Nera),Италия,белое,сухое,0.75 л.,La Scolca,Пьемонт,кортезе 100%,2022 г.,5 990
2,Красностоп Золотовский на Террасах,Россия,красное,сухое,0.75 л.,Сикоры,Кубань,красностоп 100%,2018 г.,5 490
3,Vermentino di Toscana,Италия,белое,сухое,0.75 л.,Fattoria del Cerro,Тоскана,верментино,2022 г.,2 490
4,Pfefferer,Италия,белое,полусухое,0.75 л.,Colterenzio,Трентино-Альто Адидже,мускат желтый 100%,2023 г.,2 490


Всего удалось собрать информацию о 5571 виде вина.